In [1]:
import os
import json
import re
from llama_index.core.schema import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SimpleDirectoryReader

In [2]:
import logging
from dotenv import load_dotenv
import os
import json
from fastembed import SparseTextEmbedding, TextEmbedding
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import PointStruct, SparseVector
from tqdm import tqdm

In [3]:
load_dotenv()

True

In [4]:
Qdrant_API_KEY = os.getenv('Qdrant_API_KEY')
Qdrant_URL = os.getenv('Qdrant_URL')
Collection_Name = os.getenv('collection_name')
qdrant_client = QdrantClient(
                            url=Qdrant_URL,
                            api_key=Qdrant_API_KEY)

In [5]:
Collection_Name

'Agentic-Automation-RAG'

In [6]:
def load_nodes():
        metadata = []
        documents = []
        payload_file = r'C:\Users\pavan\Desktop\Generative AI\RAG-Automation-Using-Llamaindex-Agents-and-Qdrant\data\nodes.json'

        try:
            with open(payload_file, 'r') as file:
                nodes = json.load(file)

            for node in nodes:
                metadata.append(node['metadata'])
                documents.append(node['text'])

            print(f"Loaded {len(nodes)} the nodes from JSON file")

        except Exception as e:
            logging.error(f"Error loading nodes from JSON file: {e}")
            raise

        return documents, metadata

In [9]:
def client_collection():
    """
    Create a collection in Qdrant vector database.
    """
    
    if not qdrant_client.collection_exists(collection_name=Collection_Name): 
        qdrant_client.create_collection(
            collection_name= Collection_Name,
            vectors_config={
                    'dense': models.VectorParams(
                        size=384,
                        distance = models.Distance.COSINE,
                    )
            },
            sparse_vectors_config={
                "sparse": models.SparseVectorParams(
                            index=models.SparseIndexParams(
                            on_disk=False,              
                        ),
                    )
                }
        )
        
    print(f"Created collection '{Collection_Name}' in Qdrant vector database.")


In [10]:
client_collection()

Created collection 'Agentic-Automation-RAG' in Qdrant vector database.


In [11]:
def create_sparse_vector(sparse_embedding_model, text):
        """
        Create a sparse vector from the text using SPLADE.
        """
        sparse_embedding_model = sparse_embedding_model
        # Generate the sparse vector using SPLADE model
        embeddings = list(sparse_embedding_model.embed([text]))[0]

        # Check if embeddings has indices and values attributes
        if hasattr(embeddings, 'indices') and hasattr(embeddings, 'values'):
            sparse_vector = models.SparseVector(
                indices=embeddings.indices.tolist(),
                values=embeddings.values.tolist()
            )
            return sparse_vector
        else:
            raise ValueError("The embeddings object does not have 'indices' and 'values' attributes.")

In [12]:
def insert_documents(embedding_model, documents, metadata):
        points = []
        embedding_model = TextEmbedding(model_name=embedding_model)
        sparse_embedding_model = SparseTextEmbedding(model_name="Qdrant/bm42-all-minilm-l6-v2-attentions")
        for i, (doc, metadata) in enumerate(tqdm(zip(documents, metadata), total=len(documents))):
            # Generate both dense and sparse embeddings
            dense_embedding = list(embedding_model.embed([doc]))[0]
            sparse_vector = create_sparse_vector(sparse_embedding_model, doc)

            # Create PointStruct
            point = models.PointStruct(
                id=i,
                vector={
                    'dense': dense_embedding.tolist(),
                    'sparse': sparse_vector,
                },
                payload={
                    'text': doc,
                    **metadata  # Include all metadata
                }
            )
            points.append(point)

        # Upsert points
        qdrant_client.upsert(
            collection_name=Collection_Name,
            points=points
        )

        print(f"Upserted {len(points)} points with dense and sparse vectors into Qdrant vector database.")

In [13]:
def indexing(embedding_model) -> None:
        """
        Index the documents into the Qdrant vector database.
        """
        print("Starting to load the nodes from JSON file")
        documents, metadata = load_nodes()
        client_collection()
        print("Creation of the Qdrant Collection is Done")
        insert_documents(embedding_model, documents, metadata)
        print("Indexing of the nodes is complete")
    
    

In [14]:
indexing(embedding_model = "sentence-transformers/all-MiniLM-L6-v2")

Starting to load the nodes from JSON file
Loaded 71 the nodes from JSON file
Created collection 'Agentic-Automation-RAG' in Qdrant vector database.
Creation of the Qdrant Collection is Done


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 71/71 [00:16<00:00,  4.25it/s]


Upserted 71 points with dense and sparse vectors into Qdrant vector database.
Indexing of the nodes is complete


In [9]:
qdrant_client.set_model("sentence-transformers/all-MiniLM-L6-v2")
# comment this line to use dense vectors only
qdrant_client.set_sparse_model("prithivida/Splade_PP_en_v1")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [10]:
qdrant_client.recreate_collection(
    collection_name="Hybrid_RAG_Collection",
    vectors_config=qdrant_client.get_fastembed_vector_params(),
    # comment this line to use dense vectors only
    sparse_vectors_config=qdrant_client.get_fastembed_sparse_vector_params(),  
)

C:\Users\pavan\AppData\Local\Temp\ipykernel_15400\4029956761.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(
INFO:httpx:HTTP Request: DELETE https://c77ac75e-3a41-4acc-98d2-c9c3eb11b5ea.us-east4-0.gcp.cloud.qdrant.io:6333/collections/Hybrid_RAG_Collection "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT https://c77ac75e-3a41-4acc-98d2-c9c3eb11b5ea.us-east4-0.gcp.cloud.qdrant.io:6333/collections/Hybrid_RAG_Collection "HTTP/1.1 200 OK"


True

In [11]:
import json

input_file = r"C:\Users\pavan\Desktop\Generative AI\RAG-Automation-Using-Llamaindex-Agents-and-Qdrant\data\nodes.json"
metadata = []
documents = []

with open(input_file, 'r') as file:
        nodes = json.load(file)

for node in nodes:
    metadata.append(node['metadata'])
    documents.append(node['text'])

In [12]:
len(documents)

71

In [13]:
from tqdm import tqdm

qdrant_client.add(
    collection_name="Hybrid_RAG_Collection",
    documents=documents,
    metadata=metadata,
    ids=tqdm(range(len(documents))),
)

  0%|          | 0/71 [00:00<?, ?it/s]INFO:httpx:HTTP Request: GET https://c77ac75e-3a41-4acc-98d2-c9c3eb11b5ea.us-east4-0.gcp.cloud.qdrant.io:6333/collections/Hybrid_RAG_Collection "HTTP/1.1 200 OK"


In [ ]:
indexing(embedding_model = 'sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
def DocumentPreprocessingAgent(state: dict) -> OpenAIAgent:

    

    
    

    def done() -> None:
        """When you inserted the vetors into the Qdrant Cluster, call this tool."""
        print("Indexing of the nodes is complete")
        state["current_speaker"] = None
        state["just_finished"] = True

    tools = [
        FunctionTool.from_defaults(fn=indexing),
        FunctionTool.from_defaults(fn=done),
    ]

    system_prompt = (f"""
    You are a helpful assistant that is indexing documents for a retrieval-augmented generation (RAG) system.
    Your task is to index the documents into a Qdrant cluster.
    To do this, you need to know the embedding model to use.
    You can ask the user to supply this.
    If the user supplies the embedding model, call the tool "indexing" with this parameter to index the documents into the Qdrant cluster.
    The current user state is:
    {pprint.pformat(state, indent=4)}
    When you have indexed the documents into the Qdrant cluster, call the tool "done" to signal that you are done.
    If the user asks to do anything other than index the documents, call the tool "done" to signal some other agent should help.
    """)

    return OpenAIAgent.from_tools(
        tools,
        llm=OpenAI(model="gpt-3.5-turbo"),
        system_prompt=system_prompt,
    )
